In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Object Recog/Yolo

/content/drive/MyDrive/Object Recog/Yolo


In [7]:
import os
os.getcwd()

'/content/drive/MyDrive/Object Recog/Yolo'

In [5]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2022-03-04 17:15:56--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  36.8MB/s    in 6.8s    

2022-03-04 17:16:03 (35.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [8]:
import numpy as np
import cv2
import subprocess
import time
import os

def show_image(img):
    cv2.imshow("Image", img)
    cv2.waitKey(0)

def draw_labels_and_boxes(img, boxes, confidences, classids, idxs, colors, labels):
    # If there are any detections
    if len(idxs) > 0:
        for i in idxs.flatten():
            # Get the bounding box coordinates
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]
            
            # Get the unique color for this class
            color = [int(c) for c in colors[classids[i]]]

            # Draw the bounding box rectangle and label on the image
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            text = "{}: {:4f}".format(labels[classids[i]], confidences[i])
            cv2.putText(img, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return img


def generate_boxes_confidences_classids(outs, height, width, tconf):
    boxes = []
    confidences = []
    classids = []

    for out in outs:
        for detection in out:
            #print (detection)
            #a = input('GO!')
            
            # Get the scores, classid, and the confidence of the prediction
            scores = detection[5:]
            classid = np.argmax(scores)
            confidence = scores[classid]
            
            # Consider only the predictions that are above a certain confidence level
            if confidence > tconf:
                # TODO Check detection
                box = detection[0:4] * np.array([width, height, width, height])
                centerX, centerY, bwidth, bheight = box.astype('int')

                # Using the center x, y coordinates to derive the top
                # and the left corner of the bounding box
                x = int(centerX - (bwidth / 2))
                y = int(centerY - (bheight / 2))

                # Append to list
                boxes.append([x, y, int(bwidth), int(bheight)])
                confidences.append(float(confidence))
                classids.append(classid)

    return boxes, confidences, classids

confidence=0.5 #default is set to 0.5
threshold=0.3 #The threshold to use when applying the Non-Max Suppresion

def infer_image(net, layer_names, height, width, img, colors, labels, 
            boxes=None, confidences=None, classids=None, idxs=None, infer=True):
    
    if infer:
        # Contructing a blob from the input image
        blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), 
                        swapRB=True, crop=False)

        # Perform a forward pass of the YOLO object detector
        net.setInput(blob)

        # Getting the outputs from the output layers
        start = time.time()
        outs = net.forward(layer_names)
        end = time.time()

        
        #print ("YOLOv3 took {:6f} seconds".format(end - start)) #Uncomment this line to print the time the model takes 


        
        # Generate the boxes, confidences, and classIDs
        boxes, confidences, classids = generate_boxes_confidences_classids(outs, height, width,confidence)
        
        # Apply Non-Maxima Suppression to suppress overlapping bounding boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences,confidence,threshold)

    if boxes is None or confidences is None or idxs is None or classids is None:
        raise 'Required variables for drawing boxes on images are set to None.'
        
    # Draw labels and boxes on the image
    img = draw_labels_and_boxes(img, boxes, confidences, classids, idxs, colors, labels)

    return img, boxes, confidences, classids, idxs


In [9]:
labels = open('/content/drive/MyDrive/Object Recog/Yolo/coco-labels').read().strip().split('\n')
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')
video_path='/content/drive/MyDrive/Object Recog/Road traffic.mp4' #The path to the video which you wish to use for detection
video_output_path='/content/drive/MyDrive/Object Recog/output.avi' 
# Load the weights and configutation to form the pretrained YOLOv3 model
net = cv2.dnn.readNetFromDarknet('/content/drive/MyDrive/Object Recog/Yolo/yolov3.cfg', '/content/drive/MyDrive/Object Recog/Yolo/yolov3.weights')


layer_names = net.getLayerNames()
layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

	
vid = cv2.VideoCapture(video_path)
height, width = None, None
writer = None
while True:
		grabbed, frame = vid.read()
	  
		if not grabbed:
			break

		if width is None or height is None:
			height, width = frame.shape[:2]
		
		frame, _, _, _, _ = infer_image(net, layer_names, height, width, frame, colors, labels)
	  
		if writer is None:
			fourcc = cv2.VideoWriter_fourcc(*"MJPG")
			writer = cv2.VideoWriter(video_output_path, fourcc, 30, 
						            (frame.shape[1], frame.shape[0]), True)

		writer.write(frame)

     


writer.release()
vid.release()

cv2.destroyAllWindows()

